In [2]:
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [3]:
# Create the agent
memory = InMemorySaver()
model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [4]:
search = TavilySearch(max_results=2)
tools = [search]

In [6]:
llm_with_tools = model.bind_tools(tools)

sys_msg = SystemMessage(
    content="Use either tools to perform arithmetic on a set of inputs or just give me answer with your knowledge")

def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}


In [7]:
# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
builder.add_edge("assistant", END)


In [8]:
memory = InMemorySaver()
react_graph = builder.compile(checkpointer=memory)

# thread is identifies the session
config = {"configurable": {"thread_id": "1"}}

In [9]:
human_message = HumanMessage("Hi, I'm Srikanth and I live in Visakhapatnam.")
response = react_graph.invoke({"messages": [human_message]}, config=config)

In [10]:
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi, I'm Srikanth and I live in Visakhapatnam.
================================== Ai Message ==================================

Hello Srikanth! It's nice to meet you. Is there anything I can help you with today regarding Visakhapatnam or anything else?


In [11]:
human_message = HumanMessage("Search for the weather where I live")
response = react_graph.invoke({"messages": [human_message]}, config=config)

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi, I'm Srikanth and I live in Visakhapatnam.
================================== Ai Message ==================================

Hello Srikanth! It's nice to meet you. Is there anything I can help you with today regarding Visakhapatnam or anything else?
================================ Human Message =================================

Search for the weather where I live
================================== Ai Message ==================================
Tool Calls:
  tavily_search (93227bcf-5134-4a5b-9677-b7ae74b6e56c)
 Call ID: 93227bcf-5134-4a5b-9677-b7ae74b6e56c
  Args:
    query: weather in Visakhapatnam
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Visakhapatnam", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Visakhapatnam", "url": "https://www.weatherapi.com/", "content": "{'locatio